In [1]:
from data.t5_dataset import T5CursorDataset
import numpy as np

dataset = T5CursorDataset('./data/t5.2021.06.07.mat')

In [2]:
dataset.resample(0.01)

In [188]:
np.all((None, None), 0)

In [187]:
train_dataset = np.empty(())
train_dataset.shape


()

In [202]:
# dataset
dataset.data[dataset.data['blockNums'].isin([14, 15]).values.squeeze()].spikes.to_numpy().shape


(52212, 192)

In [196]:
trial_mask = np.all(
    (
        ~dataset.trial_info['is_successful'],  # successful trials
        dataset.trial_info['is_center_target'],  # not center target
    ),
    axis=0
)

In [155]:
print(dataset.trial_info['is_successful'].to_numpy().shape)
print(dataset.trial_info['is_center_target'].to_numpy().shape)

(536,)
(536,)


In [31]:
tdata = dataset.make_trial_data(
    align_field='start_time',
    align_range=(-290, 2000),
    allow_overlap=True,
    ignored_trials=(np.all((None, None), 0))
)
tdata.sort_index(axis=1, inplace=True)

Allowed 315 overlapping trials.


In [33]:
test = [3]
test3 = [33]

test+test3

[3, 33]

In [32]:
tdata

signal_type               align_time blockNums             clock_time  \
channel                                      n                          
0           -1 days +23:59:59.980000        14        0 days 00:00:00   
1           -1 days +23:59:59.990000        14 0 days 00:00:00.010000   
2                    0 days 00:00:00        14 0 days 00:00:00.020000   
3             0 days 00:00:00.010000        14 0 days 00:00:00.030000   
4             0 days 00:00:00.020000        14 0 days 00:00:00.040000   
...                              ...       ...                    ...   
122647        0 days 00:00:01.360000        17 0 days 00:18:35.330000   
122648        0 days 00:00:01.370000        17 0 days 00:18:35.340000   
122649        0 days 00:00:01.380000        17 0 days 00:18:35.350000   
122650        0 days 00:00:01.390000        17 0 days 00:18:35.360000   
122651        0 days 00:00:01.400000        17 0 days 00:18:35.370000   

signal_type   cursorPos            cursorSize decClick    decVel            \
channel               x          y       size    state         x         y   
0              0.000000   0.000000         25        0  0.000000  0.000000   
1              0.000000   0.000000         25        0  0.000000  0.000000   
2              0.000000   0.000000         25        0  0.000000  0.000000   
3              0.000000   0.000000         25        0  0.000000  0.000000   
4              0.000000   0.000000         25        0  0.000000  0.000000   
...                 ...        ...        ...      ...       ...       ...   
122647      -352.124054 -18.037422         25        2 -0.000060  0.000031   
122648      -354.789062 -16.586653         25        2 -0.000059  0.000033   
122649      -357.343536 -15.080162         25        2 -0.000054  0.000034   
122650      -359.753021 -13.503881         25        2 -0.000053  0.000037   
122651      -362.066681 -11.819237         25        2 -0.000050  0.000037   

signal_type margin  ... spikes                         targetPos     \
channel             ...  ch187 ch188 ch189 ch190 ch191         x  y   
0            False  ...      0     0     0     0     0         0  0   
1            False  ...      0     0     0     0     0         0  0   
2            False  ...      1     0     1     0     0         0  0   
3            False  ...      0     1     1     1     0         0  0   
4            False  ...      0     0     0     0     0         0  0   
...            ...  ...    ...   ...   ...   ...   ...       ... ..   
122647       False  ...      1     0     0     0     0      -409  0   
122648       False  ...      0     0     1     0     0      -409  0   
122649       False  ...      1     1     1     1     1      -409  0   
122650       False  ...      0     0     2     0     2      -409  0   
122651       False  ...      0     0     0     0     0      -409  0   

signal_type targetSize trial_id               trial_time  
channel           size                                    
0                 57.5        0 -1 days +23:59:59.980000  
1                 57.5        0 -1 days +23:59:59.990000  
2                 57.5        0          0 days 00:00:00  
3                 57.5        0   0 days 00:00:00.010000  
4                 57.5        0   0 days 00:00:00.020000  
...                ...      ...                      ...  
122647            85.0      535   0 days 00:00:01.360000  
122648            85.0      535   0 days 00:00:01.370000  
122649            85.0      535   0 days 00:00:01.380000  
122650            85.0      535   0 days 00:00:01.390000  
122651            85.0      535   0 days 00:00:01.400000  

[122652 rows x 208 columns]

In [136]:
tdata['X&Y'] = list(zip(tdata['targetPos']['x'], tdata['targetPos']['y']))

conds = []
for xy in (tdata['X&Y'].values):
    if xy not in conds:
        conds.append(xy)

ids = np.arange(1,9)
conds_and_ids = list(zip(conds, ids))

#add conditions to target data
tdata['condition'] = 0

for xy, id in conds_and_ids:    
    indices = tdata.index[tdata['X&Y'] == xy]
    tdata.loc[indices, 'condition'] = id

In [194]:
ol_mask = tdata['blockNums'].isin([14]).values.squeeze()
cl_mask = tdata['blockNums'].isin([15,16,17]).values.squeeze()

In [195]:
tdata[ol_mask].spikes

channel,ch000,ch001,ch002,ch003,ch004,ch005,ch006,ch007,ch008,ch009,...,ch182,ch183,ch184,ch185,ch186,ch187,ch188,ch189,ch190,ch191
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,2,0,0,0,...,0,0,1,1,1,1,0,1,0,0
3,0,0,0,0,0,2,2,1,0,0,...,0,0,1,1,0,0,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23811,1,0,1,1,3,0,1,0,0,2,...,2,1,0,0,0,0,0,1,0,0
23812,0,0,0,0,0,0,0,0,0,1,...,3,0,0,0,0,0,1,0,0,2
23813,0,0,0,0,0,0,1,0,0,0,...,1,1,1,1,1,1,1,0,0,2
23814,0,0,0,0,0,1,1,1,0,1,...,1,2,0,0,0,0,1,0,1,0


In [139]:
import os.path as osp
import os
import h5py
from nlb_tools.make_tensors import save_to_h5, h5_to_dict

In [140]:
h5_file = {'real_test':{'test': tdata[ol_mask].spikes.to_numpy()}}
filename = f'data/test.h5'

# Remove older version if it exists
if osp.isfile(filename): os.remove(filename)
save_to_h5(h5_file, filename, overwrite=True)

In [141]:
with h5py.File(filename, 'r') as h5file:
    h5dict = h5_to_dict(h5file)

In [142]:
h5dict

{'real_test': {'test': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 2],
         [0, 0, 0, ..., 0, 1, 0],
         [1, 1, 0, ..., 0, 0, 0]], dtype=uint8)}}

In [159]:
lag_bins = 6
overlap = 25
chopsize = 30

In [ ]:
def chop_trial(trial):
    if lag_bins > 0:
        trial = trial[:-lag_bins, :]
    shape = (
        int((trial.shape[0] - overlap) / (chopsize - overlap)),
        chopsize,
        trial.shape[-1],
    )
    strides = (
        trial.strides[0] * (chopsize - overlap),
        trial.strides[0],
        trial.strides[1],
    )
    return np.lib.stride_tricks.as_strided(trial, shape=shape, strides=strides).copy().astype('f')

In [158]:
for tid, trial in tdata.groupby('trial_id'):
    if trial.spikes.to_numpy().shape[0] != 235:
        print(tid)

0
534
535
